In [1]:
pwd

'/home/ubuntu/astromer/presentation/notebooks'

In [2]:
cd ../..

/home/ubuntu/astromer


In [3]:
# Import required modules
from src.data.record import DataPipeline
import tensorflow as tf

import pandas as pd
import numpy as np
from os.path import join
import polars as pl
import os
from glob import glob
import sys
# sys.path.append('/home/Samsung2TB/')
# sys.path.append('/home/Samsung2TB/astromer/')
from tqdm import tqdm
import os
from joblib import Parallel, delayed
from src.data.record import DataPipeline
import tensorflow as tf

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from time import time
import logging 
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

In [4]:
# Define local path
local_path = '/home/ubuntu/astromer/data/raw_data/alcock/'

# Define Metadata and LCDIR paths
METAPATH = join(local_path, 'new_metadata.parquet')
LCDIR =  join(local_path, 'parquets/')

In [5]:
# Read metadata
metadata = pd.read_parquet(METAPATH)
metadata


,ID,Class,Path,Band,newID,Label
0,1.3319.10,LPV,1.3319.10.dat,1.0,0,3
1,1.3441.15,Cep_0,1.3441.15.dat,1.0,1,0
2,1.3441.25,LPV,1.3441.25.dat,1.0,2,3
3,1.3441.45,Cep_0,1.3441.45.dat,1.0,3,0
4,1.3441.1031,RRab,1.3441.1031.dat,1.0,4,4
...,...,...,...,...,...,...
21439,9.5608.870,RRab,9.5608.870.dat,1.0,21439,4
21440,9.5608.946,RRab,9.5608.946.dat,1.0,21440,4
21441,9.5609.22,EC,9.5609.22.dat,1.0,21441,2
21442,9.5609.790,RRab,9.5609.790.dat,1.0,21442,4


In [6]:
# Process metadata
metadata['Class'] = pd.Categorical(metadata['Class'])
metadata['Label'] = metadata['Class'].cat.codes
metadata['Path'] = metadata['Path'].apply(lambda x: os.path.join(LCDIR, x)) 

In [7]:
# Sample the metadata
metadata_sample = metadata.copy()

In [8]:
metadata_sample

,ID,Class,Path,Band,newID,Label
0,1.3319.10,LPV,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,0,3
1,1.3441.15,Cep_0,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,1,0
2,1.3441.25,LPV,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,2,3
3,1.3441.45,Cep_0,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,3,0
4,1.3441.1031,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,4,4
...,...,...,...,...,...,...
21439,9.5608.870,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21439,4
21440,9.5608.946,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21440,4
21441,9.5609.22,EC,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21441,2
21442,9.5609.790,RRab,/home/ubuntu/astromer/data/raw_data/alcock/par...,1.0,21442,4


In [9]:
# Create an instance of DataPipeline
myPipeline = DataPipeline(metadata=metadata_sample, 
                            context_features=['ID', 'Label', 'Class'],
                            sequential_features=['mjd', 'mag'],)


[INFO] 21444 samples loaded


In [10]:

# Prepare for k-fold cross-validation
test_metadata = metadata_sample.sample(n=1000)
k_folds = 2
# Split the data into training, validation, and test sets
myPipeline.train_val_test(val_frac=0.2, 
                        test_meta=[test_metadata], 
                        k_fold=k_folds)


[INFO] Using ID col as sample identifier
[INFO] Shuffling
[INFO] Shuffling


In [11]:
# Save the configuration into a .toml file
config_file = './config.toml'
myPipeline.write_config(config_file)

2023-06-21 13:42:28,802 - INFO - Starting the writing process for the config file.
2023-06-21 13:42:28,804 - INFO - Config file written successfully to ./config.toml


In [12]:
# Create TFRecord files and measure time taken
a = time()
var = myPipeline.run(LCDIR, METAPATH, n_jobs=2,)
b = time()
print(b-a)

  0%|          | 0/2 [00:00<?, ?it/s]2023-06-21 13:42:30,130 - INFO - Reading parquet files
2023-06-21 13:42:30,158 - INFO - Processing train subset_0
Writting test fold 1:   0%|          | 0/2 [00:00<?, ?it/s]    2023-06-21 13:42:30,206 - INFO - Finished execution of DataPipeline operations
2023-06-21 13:42:30,207 - INFO - Starting the writing process for the config file.
2023-06-21 13:42:30,208 - INFO - Config file written successfully to ./config.toml
2023-06-21 13:42:30,208 - INFO - Config file written
Writting test fold 1:   0%|          | 0/2 [00:00<?, ?it/s]

0.08166670799255371


In [13]:
var

In [13]:
# Now let's read back the records and process them
record_file = "./records/output/"
# Read the .record file
raw_dataset = myPipeline.open_and_read_record(record_file)
raw_dataset

2023-06-21 13:42:42,726 - INFO - Starting to read the file from ./records/output/.


2023-06-21 13:42:42.773458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 13:42:42.782311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 13:42:42.782899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 13:42:42.783933: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [14]:
import glob

record_files = glob.glob('/home/ubuntu/astromer/records/output/*.record')
raw_dataset = tf.data.TFRecordDataset(record_files)


In [15]:
from src.data.record import deserialize
processed_data = []
for raw_record in raw_dataset:
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    processed_data.append(deserialize(example.SerializeToString()))

2023-06-21 13:42:51,636 - INFO - Reading config from file.
2023-06-21 13:42:51,637 - INFO - Config read successfully.
2023-06-21 13:42:51.642214: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.


Key: mjd
Sequence: SparseTensor(indices=tf.Tensor([], shape=(0, 2), dtype=int64), values=tf.Tensor([], shape=(0,), dtype=float32), dense_shape=tf.Tensor([0 0], shape=(2,), dtype=int64))
Dense sequence: []
Casted sequence: []
Key: mag
Sequence: SparseTensor(indices=tf.Tensor([], shape=(0, 2), dtype=int64), values=tf.Tensor([], shape=(0,), dtype=float32), dense_shape=tf.Tensor([0 0], shape=(2,), dtype=int64))
Dense sequence: []
Casted sequence: []
[<tf.Tensor: shape=(0, 0), dtype=float32, numpy=array([], shape=(0, 0), dtype=float32)>, <tf.Tensor: shape=(0, 0), dtype=float32, numpy=array([], shape=(0, 0), dtype=float32)>]


InvalidArgumentError: slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/